In [227]:
import warnings

warnings.filterwarnings('ignore')

In [228]:
import math
import torch
import numpy as np
import torch.nn as nn


In [229]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [230]:
fletch_X, fletch_y = fetch_california_housing(return_X_y=True, as_frame=True)

In [231]:
X_train, X_test, y_train, y_test = train_test_split(fletch_X.to_numpy(), fletch_y.to_numpy(), test_size = 0.25, random_state = 13)

In [232]:
class MyDataset(torch.utils.data.Dataset):
  
    def __init__(self, X, y):
        self.X = torch.Tensor(X)
        self.y = torch.from_numpy(y).float()

    def __len__(self):
        return self.X.shape[0]
  
    def __getitem__(self, index):
        return (self.X[index], self.y[index])

In [233]:
data_train = MyDataset(X_train, y_train)
data_test = MyDataset(X_test, y_test)

print(len(data_train))
print(len(data_test))

15480
5160


In [265]:
train_loader = torch.utils.data.DataLoader(data_train,
                          batch_size=30,
                          shuffle=False)

In [266]:
test_loader = torch.utils.data.DataLoader(data_test,
                          batch_size=30,
                          shuffle=False)

In [267]:
print(len(train_loader))
print(len(test_loader))

516
172


In [268]:
import torch.nn.functional as F

In [269]:
class MyModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 3*hidden_dim)
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm1d(3*hidden_dim)
        self.dp = nn.Dropout(0.1)
        self.fc2 = nn.Linear(3*hidden_dim, output_dim)
        
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.bn(x)
        x = self.dp(x)
        x = self.fc2(x)
        return x


    def predict(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.bn(x)
        x = self.dp(x)
        x = self.fc2(x)
        return x

In [270]:
net = MyModel(8, 100, 1)
print(net)

MyModel(
  (fc1): Linear(in_features=8, out_features=300, bias=True)
  (relu): ReLU()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp): Dropout(p=0.1, inplace=False)
  (fc2): Linear(in_features=300, out_features=1, bias=True)
)


In [271]:
optimizer_SGD = torch.optim.SGD(net.parameters(), lr=1e-4)
optimizer_Adam = torch.optim.Adam(net.parameters(), lr=1e-4)
optimizer_RMSprop = torch.optim.RMSprop(net.parameters(), lr=1e-4)
criterion = nn.MSELoss()

In [272]:
optimizers = [['SGD', optimizer_SGD], ['Adam', optimizer_Adam], ['RMSprop', optimizer_RMSprop]]

In [273]:
import pandas as pd
from sklearn.metrics import r2_score

In [274]:
data_opt = pd.DataFrame(columns=['opt', 'score','epoch'])

In [277]:
num_epochs = 10

for name, opt in optimizers:
    for epoch in range(num_epochs):  
        running_loss, running_items = 0.0, 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data[0], data[1]
            labels = labels.reshape(-1, 1)

            opt.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            opt.step()

            running_loss += loss.item()
            running_items += len(labels)

            if i % 300 == 0:    
                net.eval()
                true_labels = np.array([labels[j][0] for j in range(len(labels))])
                pred_labels = np.array([outputs.detach().numpy()[j][0] for j in range(len(outputs))])

                print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                      f'Step [{i + 1}/{len(train_loader)}]. ' \
                      f'Loss: {running_loss / running_items:.3f}. ' \
                      f'R2: {r2_score(true_labels, pred_labels):.3f}', end='. ')

                test_running_total = 0.0
                for i, data in enumerate(test_loader):            
                    test_outputs = net(data[0])
                    test_labels = data[1].reshape(-1, 1)
                    test_running_total += len(data[1])

                    test_true_labels = np.array([test_labels[j][0] for j in range(len(test_labels))])
                    test_pred_labels = np.array([test_outputs.detach().numpy()[j][0] for j in range(len(test_outputs))])

                score = r2_score(test_true_labels, test_pred_labels)
                data_opt = data_opt.append({'opt': name, 'score': score, 'epoch': epoch}, ignore_index=True)
                print(f'Test R2: {r2_score(test_true_labels, test_pred_labels):.3f}')
                net.train()


print('Training is finished!')

Epoch [1/10]. Step [1/516]. Loss: 0.014. R2: 0.746. Test R2: 0.477
Epoch [1/10]. Step [301/516]. Loss: 0.035. R2: 0.099. Test R2: 0.241
Epoch [2/10]. Step [1/516]. Loss: 0.035. R2: 0.358. Test R2: 0.210
Epoch [2/10]. Step [301/516]. Loss: 0.035. R2: 0.207. Test R2: 0.243
Epoch [3/10]. Step [1/516]. Loss: 0.034. R2: 0.379. Test R2: 0.311
Epoch [3/10]. Step [301/516]. Loss: 0.034. R2: 0.157. Test R2: -0.006
Epoch [4/10]. Step [1/516]. Loss: 0.026. R2: 0.527. Test R2: 0.284
Epoch [4/10]. Step [301/516]. Loss: 0.032. R2: 0.085. Test R2: 0.212
Epoch [5/10]. Step [1/516]. Loss: 0.031. R2: 0.441. Test R2: 0.208
Epoch [5/10]. Step [301/516]. Loss: 0.033. R2: 0.146. Test R2: -0.041
Epoch [6/10]. Step [1/516]. Loss: 0.028. R2: 0.495. Test R2: 0.176
Epoch [6/10]. Step [301/516]. Loss: 0.032. R2: 0.160. Test R2: 0.378
Epoch [7/10]. Step [1/516]. Loss: 0.033. R2: 0.396. Test R2: 0.343
Epoch [7/10]. Step [301/516]. Loss: 0.033. R2: 0.129. Test R2: 0.245
Epoch [8/10]. Step [1/516]. Loss: 0.028. R2: 0

In [278]:
data_opt.sort_values('score', ascending=False)

,opt,score,epoch
53,RMSprop,0.825551,6
41,RMSprop,0.809947,0
57,RMSprop,0.809853,8
35,Adam,0.808281,7
39,Adam,0.799782,9
...,...,...,...
4,SGD,-1.380542,2
3,SGD,-1.623613,1
2,SGD,-1.911747,1
1,SGD,-2.018879,0
